In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from lightgbm import LGBMRegressor 
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from skgbm.xai import AXIL

# Dataset

In [7]:
#X, y = make_regression(n_samples=200)
#X_train, X_test, y_train, y_test = train_test_split(X, y)

In [41]:
# https://www.kaggle.com/datasets/sudalairajkumar/undata-country-profiles
df = pd.read_csv('archive/country_profile_variables.csv')

In [42]:
# https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd
df.columns = \
    df.columns.str.strip().str.lower() \
        .str.replace('\(.*\)', '') \
        .str.strip() \
        .str.replace(' ', '_') \
        .str.replace(',', '') \
        .str.replace('.', '') \
        .str.replace(':', '') 

C:\Users\krzysztof.joachimiak\AppData\Local\Temp\ipykernel_7612\1231718614.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns.str.strip().str.lower() \
C:\Users\krzysztof.joachimiak\AppData\Local\Temp\ipykernel_7612\1231718614.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns.str.strip().str.lower() \


In [43]:
TARGET = 'gdp_gross_domestic_product'
X, y = df.iloc[:, 2:].drop(columns=TARGET), df[TARGET]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [60]:
#X.convert_dtypes(convert_string=True).astype(float)

ValueError: could not convert string to float: '~0'

In [55]:
X.economy_agriculture

0      23.3
1      22.4
2      12.2
3       -99
4       0.5
       ... 
224     -99
225     -99
226    14.7
227     8.2
228    13.0
Name: economy_agriculture, Length: 229, dtype: object

# Training

AXIL only explains regressors. Additionaly, for the moment the only accepted loss function is RMSE and its variants (MSE etc.).

In [46]:
gbm_reg = LGBMRegressor().fit(X_train, y_train)
# gbm_reg = GradientBoostingRegressor().fit(X_train, y_train)
# gbm_reg = CatBoostRegressor().fit(X_train, y_train)
# gbm_reg = XGBRegressor().fit(X_train, y_train)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: surface_area, gdp_growth_rate, economy_agriculture, employment_agriculture, employment_industry, employment_services, unemployment, labour_force_participation, international_trade_exports, international_trade_imports, international_trade_balance, balance_of_payments_current_account, population_growth_rate, urban_population_growth_rate, fertility_rate_total, life_expectancy_at_birth, population_age_distribution, international_migrant_stock, refugees_and_others_of_concern_to_unhcr, infant_mortality_rate_(per_1000_live_births, health_physicians, education_government_expenditure, education_primary_gross_enrol_ratio, education_secondary_gross_enrol_ratio, education_tertiary_gross_enrol_ratio, mobile-cellular_subscriptions, mobile-cellular_subscriptions_1, threatened_species, forested_area, energy_supply_per_capita, pop_using_improved_drinking_water, pop_using_improved_san

# Fitting AXIL

In [9]:
axil = AXIL(gbm_reg)
axil.fit(X_train, y_train)

AXIL(estimator=LGBMRegressor())

In [10]:
k_test = axil.transform(X_test)
y_pred = lgb_regressor.predict(X_test)
np.isclose(y_pred, k_test.T @ y_train, rtol=0.000001).all()

True

# Visualization

## Heatmap

## Clustermap

## Network